<h1>Toxic Comments Analysis</h1>
<ul>
    <li><a href="#1" style="text-decoration: none;">Why we are working about?</a></li>
    <li><a href="#2" style="text-decoration: none;">What data we used?</a></li>
    <li>
        <ul>
            <li><a href="#3" style="text-decoration: none;">Load Python libray</a></li>
            <li><a href="#4" style="text-decoration: none;">Data</a></li>
            <li><a href="#5" style="text-decoration: none;">Preprocess and Visualization</a></li>
            <li><a href="#6" style="text-decoration: none;">Text Preprocess</a></li>
            <li><a href="#7" style="text-decoration: none;">Feature Extraction</a></li>
            <li><a href="#8" style="text-decoration: none;">Prediction</a></li>
            <li><a href="#9" style="text-decoration: none;">Evaluation</a></li>
        </ul>
    </li>
</ul>

<h3 id='1'>Why we are predicting?</h3>
<p>Discussing things you care about can be difficult. The threat of abuse and harassment online means that many people stop expressing themselves and give up on seeking different opinions. Platforms struggle to effectively facilitate conversations, leading many communities to limit or completely shut down user comments.</p>

<h3 id='2'>What data is avaiable</h3>
<p>To analys the toxic comment, we have used <a href="https://www.kaggle.com/c/nlp-getting-started/data">Kaggle Dataset</a>.</p>

<h3 id='3'>Load Python libray</h3>

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from lightgbm import LGBMClassifier
import pandas as pd
import re

<h3 id='4'>Load Data</h3>

In [17]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [18]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [19]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [20]:
print(train_df.shape)
print(test_df.shape)

(7613, 5)
(3263, 4)


<h3 id='5'>Preprocess and pre-analysis</h3>

In [23]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [24]:
train_df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [26]:
train_df = train_df.drop(['id','keyword','location'], axis=1)
test_df = test_df.text

<h3 id='6'>Text Preprocess</h3>

In [29]:
# Create small function to clean text
def text_clean(text):

    for element in ["http\S+", "RT ", "[^a-zA-Z\'\.\,\d\s]", "[0-9]","\t", "\n", "\s+", "<.*?>"]:
        text = re.sub("r"+element, " ", text)

    return text

# Clean data sets
train_df.text = train_df.text.apply(text_clean)
test_df.text = val_df.text.apply(text_clean)
print('Done')

Done


In [30]:
# CORRECT SPELLING
# Instantiate spell checker
spell = SpellChecker()

# Correct spelling
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
# Spellcheck data sets
train_df.text = train_df.text.apply(correct_spellings)
test_df.text = val_df.text.apply(correct_spellings)
print('Done')

Done


In [32]:
train_df.head()

,text,target
0,Ou Deeds are the Reason of this earthquake May...,1
1,Forest fire nea La Ronge Sask. Canada,1
2,All residents asked to shelter in place are be...,1
3,"13,000 people receive wildfires evacuation ord...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [65]:
X = train_df.text
y = train_df.target

<h3 id='7'>Feature Extraction</h3>

In [66]:
vectorizer = TfidfVectorizer(max_features=1500, min_df = 5, max_df=0.7, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(X).toarray()

<h3 id='8'>Train and Prediction</h3>

In [67]:
# Split data into train and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state = 42)

clf = LGBMClassifier()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

<h3 id='9'>Evaluation</h3>

In [72]:
accuracy = accuracy_score(y_pred, y_test)
print('LightGBM Accuracy Score: ',accuracy)

LightGBM Accuracy Score:  0.7800393959290873


In [73]:
con_met = confusion_matrix(y_pred, y_test)
con_met

array([[752, 213],
       [122, 436]], dtype=int64)

In [69]:
y_pred_train=clf.predict(X_train)
accuracy_train = accuracy_score(y_pred_train, y_train)
print('LightGBM Accuracy Score: ',accuracy_train)

LightGBM Accuracy Score:  0.8596059113300493


In [70]:
print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))

Training set score: 0.8596
Test set score: 0.7800


In [77]:
from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.86      0.78      0.82       965
           1       0.67      0.78      0.72       558

    accuracy                           0.78      1523
   macro avg       0.77      0.78      0.77      1523
weighted avg       0.79      0.78      0.78      1523

